# Parking Lots Setup

In [1002]:
import os
import pandas as pd
import numpy as np
import json

data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')

In [1003]:
df = pd.read_csv(datapath, sep=';')
print(df.shape)
# df = df.loc[~df.METERHEAD.isin(['Single / Disability', 'Single Motorbike', 'Twin / Disability'])]
df = df.loc[df.METERHEAD.isin(['Single'])]
print(df.shape)
df.head()

(7954, 21)
(618, 21)


,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,TIMEINEFFE,T_MF_9A_6P,...,T_SA_9A_6P,T_SA_6P_10,T_SU_9A_6P,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID
3,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,66055,"{""coordinates"": [-123.12590142819074, 49.28138...",West End,640925
5,Single,$5.00,$3.00,$5.00,$3.00,$5.00,$3.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,66859,"{""coordinates"": [-123.1208487679332, 49.273572...",Downtown,160120
30,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,61288,"{""coordinates"": [-123.12297806641084, 49.27807...",Downtown,601023
48,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,53777,"{""coordinates"": [-123.21091545808456, 49.26378...",West Point Grey,D04534
63,Single,$4.00,$1.00,$4.00,$1.00,$4.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,54307,"{""coordinates"": [-123.10791776294424, 49.26660...",Mount Pleasant,B50121


In [1004]:
from scipy.stats import poisson, expon

'''
A spot can be - single
Can have subclasses - Single, Twin, Station (5)
- https://towardsdatascience.com/the-poisson-distribution-and-poisson-process-explained-4e2cb17d459
'''
class ParkingSpot(object):
    
    '''
    EVENT - car leaves
    RATE - minute basis
    '''
    def __init__(self, location, geo_area, rate, time_limit, effect_duration=('9:00AM', '10:00PM')):
        self.lat, self.lon = location['lat'], location['lon']
        self.location = location
        self.geo_area = geo_area
        
        self.meter_rate = rate/60 #Adjust for hourly rate
        self.time_limit = time_limit*60 #Adjust for hour
        
        self.capacity = None #Might change depending on #stations (twin or not)
        #Rate of departure/arrival of cars (per minute)
        self.lam_depart = None
        self.lam_arrive = None
    
    def form_models(self):
        assert self.lam_depart 
        assert self.lam_arrive
        self.arrival_model = poisson(self.lam_arrive)
        self.departure_model = poisson(self.lam_depart)
    
    def get_n_departures(self, num_minutes):
        return int(self.lam_depart * num_minutes)
    
    def get_n_arrivals(self, num_minutes):
        return  int(self.lam_arrive * num_minutes)
    
    def get_expected_state(self, num_minutes):
        n_out = self.get_n_departures(num_minutes)
        n_in = self.get_n_arrivals(num_minutes)
        print(f'out: {n_out} - in: {n_in}')
        flow = n_out - n_in #Positive flow means more cars out than in
        # return  min(flow, self.capacity) if flow > 0 else 0
        return self.capacity + flow if flow > 0 else 0
    
    def get_time_between_arrivals(self):
        return float(expon.stats(scale=1/self.lam_arrive, moments='m'))
    
    def get_time_between_departures(self):
        return float(expon.stats(scale=1/self.lam_depart, moments='m'))
        
    
class SingleSpot(ParkingSpot):
    def __init__(self, location, geo_area, rate, time_limit):
        super().__init__(location, geo_area, rate, time_limit)
        self.capacity = np.random.randint(1,4)
        
        self.lam_depart = 1/np.random.randint(10, 15) # 1 car leaves every A...X minutes
        self.lam_arrive = 1/np.random.randint(15, 20+1) # 1 car arrives every A...X minutes
        self.form_models()

In [1005]:
meter = df.loc[3]
meter

METERHEAD                                                    Single
R_MF_9A_6P                                                    $1.00
R_MF_6P_10                                                    $1.00
R_SA_9A_6P                                                    $1.00
R_SA_6P_10                                                    $1.00
R_SU_9A_6P                                                    $1.00
R_SU_6P_10                                                    $1.00
RATE_MISC                                                       NaN
TIMEINEFFE                     METER IN EFFECT: 9:00 AM TO 10:00 PM
T_MF_9A_6P                                                     2 Hr
T_MF_6P_10                                                     4 Hr
T_SA_9A_6P                                                     2 Hr
T_SA_6P_10                                                     4 Hr
T_SU_9A_6P                                                     2 Hr
T_SU_6P_10                                      

In [1006]:
location = json.loads(meter['Geom'])['coordinates']
time_limit = int(meter['T_MF_9A_6P'].split(" ")[0])
rate = float(meter['R_MF_9A_6P'].split("$")[1])
print(f'Time limit: {time_limit} \t Rate: {rate}')

spot = SingleSpot(location = {'lat':location[1], 'lon':location[0]} ,
                  geo_area = meter['Geo Local Area'], 
                  rate = rate, time_limit=time_limit)
print(spot.get_expected_state(2*60))
spot.get_time_between_arrivals()
spot.get_time_between_departures()

Time limit: 2 	 Rate: 1.0
out: 12 - in: 7
6


10.0

# Draft Algorithm

- Pick a parking spot
- Find Nearest spots
- Formulate objective

In [1007]:
def get_latln(row):
    geom = json.loads(row)
    return geom['coordinates'][1], geom['coordinates'][0]


df['latln'] = df['Geom'].apply(get_latln)

In [1008]:
spot.location

{'lat': 49.2813874505898, 'lon': -123.12590142819074}

In [1009]:
from utils.tom_api import *
import geopy.distance as gd

destination = (spot.location['lat'], spot.location['lon'])
# ttapi = TomSearchApi()
def compute_dist_from_dest(row):
    return gd.geodesic(destination, row).km

df['dist_from_origin'] = df['latln'].apply(compute_dist_from_dest)

In [1010]:
# nearest_locs  = df.dist_from_origin.sort_values().index #PROPER
nearest_locs  = ((df.dist_from_origin.sort_values() >= 5.5) & (df.dist_from_origin.sort_values() <= 7.5)).sample(n=30).index
# nearest_locs = df.sample(n=20).index
nearest = df.loc[nearest_locs]

In [1011]:
#User specs
stay_duration = 1.5 * 60 #In minutes
p_budget = 8.00 #dollars

In [1012]:
from config import *
ttapi = TomSearchAPI(API_KEY)

nearest.reset_index(inplace=True)

# variables = {f'y_{i+1}':{f'b_{i+1}':None,
#                          'constraint_coeffs':None} for i in nearest.index}

variables = {}
curr_i = 1

#Iterate over all meters
for i, meter in nearest.iterrows():
    print(f"IDX - {i}")
    location = meter.latln
    time_limit = int(meter['T_MF_9A_6P'].split(" ")[0])
    rate = float(meter['R_MF_9A_6P'].split("$")[1])
    print(f'Time limit: {time_limit} \t Rate: {rate}')
    spot = SingleSpot(location = {'lat':location[0],
                                  'lon':location[1]} ,
                      geo_area = meter['Geo Local Area'], 
                      rate = rate, time_limit=time_limit)
    # print(spot.location)
    #FILTER OUT
    parking_fee = stay_duration * spot.meter_rate
    if parking_fee > p_budget:
        print("Skipping due to fee constraint")
        continue
    if stay_duration > spot.time_limit:
        print("Skipping due to time limit")
        continue
        
    try:
        travel_et = ttapi.calculate_travel_time(HOME, spot.location, travel_mode='car')['travelTimeInSeconds']/60
        walk_time = round(ttapi.calculate_travel_time(spot.location, {'lat':destination[0], 'lon':destination[1]}, travel_mode='pedestrian')['travelTimeInSeconds']/60, 3)
        print(f'*** Travel time: {travel_et} ***')
        expected_status = spot.get_expected_state(travel_et)
        print(f'~~~ Parking Status: {expected_status} ~~~')

        if expected_status < 1:
            print("SKIPPING! No spot will be available")
            continue

        curr_key = f'y_{curr_i}'
        variables[curr_key] = {}
        '''
        Coefficient b_i: Time to get to spot + time for spot to open up
        '''
        # variables[curr_key][f'b_{curr_i}'] = round(travel_et + spot.get_time_between_departures()-spot.get_time_between_arrivals(), 3)
        variables[curr_key][f'b_{curr_i}'] = round(spot.get_time_between_departures()-spot.get_time_between_arrivals(), 3)
        
        # variables[curr_key][f'b_{curr_i}'] += walk_time
        
        variables[curr_key][f'b_{curr_i}'] = round(variables[curr_key][f'b_{curr_i}'], 3)

        '''
        CONSTRAINTS DICT
        '''
        constraint_coeffs = {}
        #Coefficients of y_i
        constraint_coeffs['parking_fee'] = (-parking_fee + p_budget)
        constraint_coeffs['expected_avail'] = expected_status - 1
        variables[curr_key]['constraint_coeffs'] = constraint_coeffs
        variables[curr_key]['meterid'] = meter.METERID
        variables[curr_key]['latln'] = meter.latln

        print(variables[f'y_{curr_i}'])
        curr_i += 1
    except:
        print(i)
        print("Error")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

IDX - 0
Time limit: 2 	 Rate: 1.0
*** Travel time: 4.65 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 1
Time limit: 3 	 Rate: 3.0
*** Travel time: 20.233333333333334 ***
out: 1 - in: 1
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 2
Time limit: 2 	 Rate: 1.0
2
Error
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
IDX - 3
Time limit: 3 	 Rate: 1.0
3
Error
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
IDX - 4
Time limit: 2 	 Rate: 2.0
*** Travel time: 15.366666666666667 ***
out: 1 - in: 0
~~~ Parking Status: 3 ~~~
{'b_1': -7.0, 'constraint_coeffs': {'parking_fee': 5.0, 'expected_avail': 2}, 'meterid': 'B70518', 'latln': (49.264931020829636, -123.11625797677617)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
IDX - 5
Time limit: 3 	 Rate: 6.0
Skipping due to fee constraint
IDX - 6
Time limit: 2 	 Rate: 3.0
*** Travel time: 18.85 ***
out: 1 - in: 1
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 7
Time limit: 2 	 Rate: 3.0
7



# PuLP

In [1013]:
variables

{'y_1': {'b_1': -7.0,
  'constraint_coeffs': {'parking_fee': 5.0, 'expected_avail': 2},
  'meterid': 'B70518',
  'latln': (49.264931020829636, -123.11625797677617)},
 'y_2': {'b_2': -7.0,
  'constraint_coeffs': {'parking_fee': 0.5, 'expected_avail': 1},
  'meterid': '191003',
  'latln': (49.28546102408643, -123.1203463564441)},
 'y_3': {'b_3': -3.0,
  'constraint_coeffs': {'parking_fee': 6.5, 'expected_avail': 2},
  'meterid': 'C40140',
  'latln': (49.25834459665412, -123.101466670212)},
 'y_4': {'b_4': -4.0,
  'constraint_coeffs': {'parking_fee': 2.0, 'expected_avail': 3},
  'meterid': '570825',
  'latln': (49.27826602004481, -123.1169068124185)},
 'y_5': {'b_5': -8.0,
  'constraint_coeffs': {'parking_fee': 6.5, 'expected_avail': 3},
  'meterid': 'C60208',
  'latln': (49.25653059011256, -123.10076251599872)},
 'y_6': {'b_6': -7.0,
  'constraint_coeffs': {'parking_fee': 3.5, 'expected_avail': 3},
  'meterid': '560417',
  'latln': (49.28176340060928, -123.10997974726216)},
 'y_7': {'b_7

In [1014]:
import pulp
from pulp import *

model = LpProblem("ParkingOptimizer", LpMinimize)

#Decision variables
var_idx = list(range(1, len(variables)+1))
lp_vars = LpVariable.dicts('y', var_idx, lowBound=0)
print(lp_vars)

#Objective function!
model += lpSum([variables[str(lp_vars[vi])][f'b_{vi}'] * lp_vars[vi] for vi in var_idx])


#Availability
for v_i in var_idx:
    model += variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail'] * lp_vars[v_i] >= 0
    
#Rates
for v_i in var_idx:
    model += variables[str(lp_vars[v_i])]['constraint_coeffs'][f'parking_fee'] * lp_vars[v_i] >= 0
    

#Probabilistic
model += lpSum([1 * lp_vars[vi] for vi in var_idx]) == 1

model

{1: y_1, 2: y_2, 3: y_3, 4: y_4, 5: y_5, 6: y_6, 7: y_7, 8: y_8, 9: y_9, 10: y_10, 11: y_11, 12: y_12}


ParkingOptimizer:
MINIMIZE
-7.0*y_1 + -7.0*y_10 + -6.0*y_11 + -7.0*y_12 + -7.0*y_2 + -3.0*y_3 + -4.0*y_4 + -8.0*y_5 + -7.0*y_6 + -8.0*y_7 + -9.0*y_8 + -8.0*y_9 + 0.0
SUBJECT TO
_C1: 2 y_1 >= 0

_C2: y_2 >= 0

_C3: 2 y_3 >= 0

_C4: 3 y_4 >= 0

_C5: 3 y_5 >= 0

_C6: 3 y_6 >= 0

_C7: 3 y_7 >= 0

_C8: 3 y_8 >= 0

_C9: y_9 >= 0

_C10: 2 y_10 >= 0

_C11: 2 y_11 >= 0

_C12: 2 y_12 >= 0

_C13: 5 y_1 >= 0

_C14: 0.5 y_2 >= 0

_C15: 6.5 y_3 >= 0

_C16: 2 y_4 >= 0

_C17: 6.5 y_5 >= 0

_C18: 3.5 y_6 >= 0

_C19: 5 y_7 >= 0

_C20: 6.5 y_8 >= 0

_C21: 6.5 y_9 >= 0

_C22: 2 y_10 >= 0

_C23: 2 y_11 >= 0

_C24: 6.5 y_12 >= 0

_C25: y_1 + y_10 + y_11 + y_12 + y_2 + y_3 + y_4 + y_5 + y_6 + y_7 + y_8 + y_9
 = 1

VARIABLES
y_1 Continuous
y_10 Continuous
y_11 Continuous
y_12 Continuous
y_2 Continuous
y_3 Continuous
y_4 Continuous
y_5 Continuous
y_6 Continuous
y_7 Continuous
y_8 Continuous
y_9 Continuous

In [1015]:
#SOLVE
model.solve()

print("*"*30 + "STATUS" + "*"*30)
print(f'Solution status: {LpStatus[model.status]}')
print("~~~ Values ~~~")
for variable in model.variables():
        print(variable.name, " =", variable.varValue)
# print(f"Maximum profit (under constraints) = ${pulp.value(lp_problem.objective)}")
print("*"*30 + "***" + "*"*30)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/883c269bad0d4bdaaed67d990143396a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/883c269bad0d4bdaaed67d990143396a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 79 RHS
At line 105 BOUNDS
At line 106 ENDATA
Problem MODEL has 25 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-25) rows, 0 (-12) columns and 0 (-36) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -9
After Postsolve, objective -9, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -9 - 0 iterations time 0.002, Presolve 0.00
Option for print

In [1016]:
for variable in model.variables():
    if variable.varValue > 0:
        print(variables[variable.name])

{'b_8': -9.0, 'constraint_coeffs': {'parking_fee': 6.5, 'expected_avail': 3}, 'meterid': 'L10633', 'latln': (49.223635025709136, -123.09137190572585)}
